#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [2]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [3]:
embeddings=OllamaEmbeddings()
db=FAISS.from_documents(docs,embeddings)
db

/var/folders/hj/8fxfbdp516z1q9b5vhdr9yq00000gn/T/ipykernel_84716/2759370084.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings=OllamaEmbeddings()


In [4]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [5]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [6]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='31deb697-35f9-492f-a421-0347ec6c716d', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(15595.176)),
 (Document(id='f40acc9c-78ed-4b21-8166-b6345eda4c60', metadata={'source': 'spe

In [7]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[1.8895891904830933,
 -3.0950088500976562,
 1.9681270122528076,
 1.4708982706069946,
 -0.8889293074607849,
 0.6570269465446472,
 1.5206317901611328,
 -0.4903864860534668,
 0.9176185727119446,
 -2.010150671005249,
 1.3014225959777832,
 -1.9611146450042725,
 -0.6220726370811462,
 1.5268915891647339,
 -0.4981527328491211,
 -2.0465896129608154,
 -0.42361634969711304,
 0.06471753865480423,
 0.7282411456108093,
 -1.83975350856781,
 -0.6957804560661316,
 -1.035378336906433,
 2.167562961578369,
 -2.0170400142669678,
 0.7551841139793396,
 -0.9736971259117126,
 0.3396073877811432,
 -1.6768742799758911,
 0.16944102942943573,
 -0.854266345500946,
 2.0832691192626953,
 -2.8527770042419434,
 -2.6644415855407715,
 3.772991895675659,
 2.2575676441192627,
 -5.105350017547607,
 -1.0416896343231201,
 1.377994418144226,
 -1.2589880228042603,
 -1.3826842308044434,
 -0.46705976128578186,
 -2.042888879776001,
 2.143576145172119,
 0.3478071689605713,
 0.9235236644744873,
 0.5517841577529907,
 -0.2570897638797

In [8]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='31deb697-35f9-492f-a421-0347ec6c716d', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='f40acc9c-78ed-4b21-8166-b6345eda4c60', metadata={'source': 'speech.txt'}, page_content='To 

In [9]:
### Saving And Loading
db.save_local("faiss_index")

In [10]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [11]:
docs

[Document(id='31deb697-35f9-492f-a421-0347ec6c716d', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='f40acc9c-78ed-4b21-8166-b6345eda4c60', metadata={'source': 'speech.txt'}, page_content='To 

In [12]:
!ls

5.1-Faiss.ipynb  chroma_db        speech.txt
5.2-Chroma.ipynb faiss_index


In [13]:
!pwd

/Users/anurag2/Desktop/udemy/Gen-AI/c. Basics+Of+Langchain/5-VectorStore
